TODO
- Predict: Class of event -> Class of next event
  - simple biased predictor
  - hierarchical bayes predictor
- Predict: All features -> Class of next event
  - decision tree
  - random forest
  - random forest adaboosted

In [1]:
import pandas as pd
import numpy as np
data_folder = "../data"
contest = "England"

df = pd.read_csv(f"{data_folder}/parsed_{contest}.csv")

print(df)

        Unnamed: 0  subEventName              tags       positions  matchId  \
0                0   Simple pass      ['Accurate']  (51.45, 33.32)  2499719   
1              901   Simple pass      ['Accurate']    (52.5, 34.0)  2499719   
2                1     High pass      ['Accurate']  (32.55, 53.04)  2499719   
3              902     High pass  ['Not accurate']   (65.1, 39.44)  2499719   
4                2     Head pass      ['Accurate']   (53.55, 51.0)  2499719   
...            ...           ...               ...             ...      ...   
643145      642431   Simple pass      ['Accurate']    (44.1, 6.12)  2500098   
643146      642432     High pass      ['Accurate']   (12.6, 31.28)  2500098   
643147      642433  Acceleration      ['Accurate']   (27.3, 61.88)  2500098   
643148      642434   Simple pass      ['Accurate']   (63.0, 65.28)  2500098   
643149      642435   Simple pass  ['Not accurate']  (43.05, 56.44)  2500098   

                 eventName  teamId matchPeriod     

In [2]:
import numpy as np
from collections import Counter
from itertools import chain

# print(len(Counter(df.matchId)))
# print(*sorted(Counter([tuple(x) for x in (df[['eventName', 'subEventName']].dropna().values)]).items(), key=lambda x: x[1], reverse=True), sep='\n')

# list(**chain.from_iterable(list(**df.tags)))
alltags = [x for x in map(eval, list(df.tags))]
# # print(Counter(chain.from_iterable(df.tags)))
# Counter(chain.from_iterable(alltags))

# 'Goal' in df.tags

# df.tags
count = 0
for tags in alltags:
    if 'Goal' in tags and 'Accurate' in tags:
        count+= 1

count
# df['Goal' in df.tags]

# Counter(alltags)
# df.columns



986

In [3]:
matches_england = pd.read_json("../data/figshare/matches_England.json")

# teamss = matches_england['teamsData']

# unformed_teams = set([teams[team]['hasFormation'] for teams in teamss for team in teams])
# unformed_teams
# unformed_teams[0]

total = 0
for label in matches_england['label']:
    label = label.split()
    a, b = int(label[-3]), int(label[-1])
    total += a + b

total


1018

Priorities

1. Probabilistic classification of the next event type/subtype/tags, given the most recent event details
2. Regression/estimation of the expected waiting time until the next event, given the most recent event details
    - most likely exponential
3. A basic recursive predictive model
4. Implementation of a Kalman filter to model ball position
5. Putting this all into a nice looking report


exponential distribution for the expected waiting time

Regression problem, what are first and second moments of the position


ideas
- look into spatio-temporal methods

William recommends making flowchart

notes:

quasi-constant variables = vairables that are mostly the same throughout

havne't done much analysis yet
written the abstract, know what we want to do
- Don't know what to predict
    - predicting winners
        - one hot encoding
        - get dummies
        - too many tags to build encoding for
        - sparse data, so support vector is ideal

A lot of categorical features. This induces a lot of variance.
- bagging sounds good to combat this

time series data, CV-split is time-sensitive

overview of the game of soccer

key step is to clean the data

we have code ready for us
- gridsearch from labs
- stuff on github
    - we can use the metrics they've built


Matthew says to eliminate constant and quasiconstant variables. 
eg. The acceleration index is almost always 0.

randomised CV search ??


structure-wise
- feature engineering
- 
- fit a basic model in
- get a metric and evaluate it

pick something that we're trying to predict/classify
- who wins
- what, when, and where the next event will be

methods
- oversampling and undersampling methods -> william's skilset. See if we can find something to predict
    - who will win
    - injuries
        - red/yellow cards?
    - 

build some simple models


predict match outcomes from previous match data
- use scores from previous matches
    - collect data from wins/losses
    - pick some simple models to apply to it
        - performance score for each game?
        - skill for each team?
- use team 
- maybe position data
- 


determine if players are playing particularly forward or backward compared to their position


figure out what the distribution is for the first event, so that we can extrapolate for the rest of the match


pretty graphs



logistic
tuned
plot roc uac

ideas

Determine which directions let players keep the ball the longest
 - quad-tree
 - not sure how to make this machine-learning, it seems like a primarily statistical thing



plan:
- catch up on what's going on
    - make notes on things to pay attention to
- (babble) make a list of things that are possible to do in a day
- prioritise and build a plan
    - flesh out steps in detail
- execute

# Analysis on past match data

In [67]:
import pandas as pd
import numpy as np
from collections import defaultdict
matches_england = pd.read_json("../data/figshare/matches_England.json")

# print(matches_england.iloc[0])

def extract_match_results(matches):
    def f(row):
        # print(row)
        teams, scores = row['label'].strip().split(',')
        t1, t2 = teams.strip().split(' - ')
        s1, s2 = map(int, scores.strip().split(' - '))
        date = row['dateutc']
        # return {'t1':t1, 't2':t2, 's1':s1, 's2':s2, 'date':date}
        return {'s1': s1, 's2': s2, 'date': date, t1: 1, t2: -1}

    results = matches.apply(f, axis=1)
    # results = pd.DataFrame(results)

    results = pd.DataFrame(list(results))
    results = results.sort_values(by='date', axis=0).reset_index()
    results = results.fillna(0)

    return results

# def one_hot_matches(results):


results = extract_match_results(matches_england)
teamnames = [team for team in results if team not in ['s1', 's2', 'date']]
# print(teamnames)

# teamnames = list(set(results.t1).union(results.t2))



# teams = set()
# for lineups in matches_england['teamsData']:
#     for team in lineups.keys():
#         teams.add(team)

# print(teams)
# matches_england

# print(matches_england['teamsData'][0])
# print(results)
# print(results.iloc[8:20,:])
for i in range(1, len(results)):
    if results['date'][i] < results['date'][i-1]:
        print(i)
    
# print(results['date'][10], results['date'][9], )

     index  s1  s2                 date  Burnley  AFC Bournemouth  \
0      379   4   3  2017-08-11 18:45:00      0.0              0.0   
1      378   3   3  2017-08-12 11:30:00      0.0              0.0   
2      376   0   0  2017-08-12 14:00:00      0.0              0.0   
3      373   2   3  2017-08-12 14:00:00     -1.0              0.0   
4      377   1   0  2017-08-12 14:00:00      0.0             -1.0   
..     ...  ..  ..                  ...      ...              ...   
375      3   4   0  2018-05-13 14:00:00      0.0              0.0   
376      2   0   1  2018-05-13 14:00:00      0.0              0.0   
377      1   2   0  2018-05-13 14:00:00      0.0              0.0   
378      9   3   1  2018-05-13 14:00:00      0.0              0.0   
379      0   1   2  2018-05-13 14:00:00      1.0             -1.0   

     Crystal Palace  West Bromwich Albion  Huddersfield Town  Arsenal  ...  \
0               0.0                   0.0                0.0      1.0  ...   
1              

In [68]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error as mse

y, X = (results['s1'] - results['s2']).values, results[teamnames].values
# print(y)
# print(X)
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] #.reshape((-1, 1))
    # print(y_train.shape)
    clf = LogisticRegression(random_state=23).fit(X_train, (y_train > 0))
    print("Logistic regression accuracy: ", clf.score(X_test, y_test > 0))
    print(clf.predict_proba([[0]*len(teamnames)]))
    # print(clf.intercept_, clf.coef_)

    # clf = LogisticRegression(random_state=23).fit(X_train, (y_train > 0))
    # print("Logistic regression accuracy: ", clf.score(X_test, y_test > 0))
    # print(clf.predict_proba([[0]*len(teamnames)]))


    lr: LinearRegression = LinearRegression().fit(X_train, y_train)
    print(mse(y_test, lr.predict(X_test)))



Logistic regression accuracy:  0.6666666666666666
[[0.67784142 0.32215858]]
2.7875103390440015
Logistic regression accuracy:  0.6825396825396826
[[0.203729 0.796271]]
3.9818811591063024
Logistic regression accuracy:  0.6349206349206349
[[0.56312827 0.43687173]]
2.663751545625254
Logistic regression accuracy:  0.6507936507936508
[[0.55584899 0.44415101]]
2.3991456926077523
Logistic regression accuracy:  0.5238095238095238
[[0.48959046 0.51040954]]
3.0482783891840044


In [70]:
# impl 
# def train_next_goal_home_prior_model(matches)

# grab match data (done?)
# train on 


def extract_teams_from_match(match):
    teams, scores = match['label'].strip().split(',')
    t1, t2 = teams.strip().split(' - ')
    s1, s2 = map(int, scores.strip().split(' - '))
    return t1, t2

# Predicts how many goals are expected of each team using a linear model, 
# and uses that to calculate the probability that the next goal is from the home team.
def train_next_goal_home_prior_model(prior_matches):
    results = extract_match_results(prior_matches)
    teamnames = sorted([team for team in results if team not in ['s1', 's2', 'date']])

    # doubling data by expressing it in terms of either team, with an extra column to track who is home
    results['home'] = 1
    y, X = results['s1'].values, results[['home'] + teamnames].values
    y2, X2 = results['s2'].values, -results[['home'] + teamnames].values
    y = pd.concat([y, y2], axis=0)
    X = pd.concat([X, X2], axis=0)

    lr: LinearRegression = LinearRegression().fit(X, y)

    def format_match(match):
        home, away = extract_teams_from_match(match)
        return np.array([1] + [(team == home) - (team == away) for team in teamnames])

    def prediction(match):
        features_home = format_match(match)
        expected_goals_home = lr.predict(features_home)
        features_away = -format_match(match)
        expected_goals_away = lr.predict(features_away)

        return expected_goals_home / (expected_goals_home + expected_goals_away)

    return prediction, lr, format_match




In [ ]:
# test train_next_goal_home_prior_model()

matches_england = pd.read_json("../data/figshare/matches_England.json")
half_matches = matches_england.iloc[:len(matches_england)//2, :]

predictor, model, formatter = train_next_goal_home_prior_model(half_matches)

